In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

In [2]:
# Load the Food101 dataset info
food101_info = tfds.builder('food101').info

In [6]:
# Load the dataset
(train_data, test_data) = tfds.load('food101', split=['train', 'validation'], as_supervised=True)

In [7]:
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = preprocess_input(image)  # Preprocessing specific to ResNet-50
    return image, label

In [8]:
# Preprocess the data
train_data = train_data.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_data.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
])

# Apply data augmentation on training dataset
train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))

In [10]:
# Initialize the ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the layers

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(101, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

94765736/94765736 [==============================] - 3s 0us/step


In [11]:
history = model.fit(train_data, epochs=5, validation_data=test_data)

Epoch 1/5
2368/2368 [==============================] - 807s 336ms/step - loss: 2.4033 - accuracy: 0.4092 - val_loss: 1.7072 - val_accuracy: 0.5477
Epoch 2/5
2368/2368 [==============================] - 798s 337ms/step - loss: 2.0010 - accuracy: 0.4929 - val_loss: 1.5597 - val_accuracy: 0.5835
Epoch 3/5
2368/2368 [==============================] - 793s 335ms/step - loss: 1.8754 - accuracy: 0.5207 - val_loss: 1.5518 - val_accuracy: 0.5884
Epoch 4/5
2368/2368 [==============================] - 796s 336ms/step - loss: 1.8081 - accuracy: 0.5359 - val_loss: 1.5228 - val_accuracy: 0.5960
Epoch 5/5
2368/2368 [==============================] - 797s 337ms/step - loss: 1.7583 - accuracy: 0.5465 - val_loss: 1.5251 - val_accuracy: 0.5927


In [12]:
loss, accuracy = model.evaluate(test_data)
print(f"Test accuracy: {accuracy * 100}%")

790/790 [==============================] - 27s 34ms/step - loss: 1.5251 - accuracy: 0.5927
Test accuracy: 59.27128791809082%
